# Imports

In [ ]:
import pandas                as pd
import numpy                 as np
import matplotlib.pyplot     as plt
import seaborn               as sns
import missingno             as msno
from sklearn.linear_model    import LinearRegression
from sklearn.linear_model    import LassoCV
from sklearn.linear_model    import RidgeCV
from sklearn.metrics         import r2_score
from sklearn.metrics         import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.pipeline        import Pipeline
from IPython.core.display    import display, HTML
sns.set(style = "white", palette = "husl")
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline

# Table Of Contents

-----

1. [Reading In The Data](#Reading-In-The-Data)
    - [Overview](#Overview)

-----

2. [Feature Engineering](#Feature-Engineering)
    - [Transforming Numeric Data](#Transforming-Numeric-Data)
    - [Creating Segmental Features](#Creating-Segmental-Features)
    - [Implementing The Pattern Sub-Model Approach](#Implementing-The-Pattern-Sub-Model-Approach)
        - [Creating The Sub-Models](#Creating-The-Sub-Models)
        - [Calculating The RMSE Weights](#Calculating-The-RMSE-Weights)
    
-----

3. [Modeling](#Modeling)
    - [Functions](#Functions)
        - [Model Functions](#Model-Functions)
        - [Evaluation Functions](#Evaluation-Functions)
    - [Linear Regression](#Linear-Regression)
    - [Ridge](#Ridge)
    - [LASSO](#LASSO)

# Reading In The Data

In [ ]:
mri = pd.read_csv("../Data/mri_cleaned.csv")

## Overview

In [ ]:
mri.head()

The `smoker_status` column is still in the data, but we will not needing the column for the models because we turned it into a pair of dummy columns so we will drop the column here.

In [ ]:
#  Dropping `smoker_status`

mri = mri.drop("smoker_status", axis = 1)

In [ ]:
# Checking the shape of the data

print(f"The shape of the dataset is: {mri.shape}")

In [ ]:
# Summary of column data types

mri.dtypes.value_counts()

In [ ]:
# Checking for columns with missing/NaN data

(mri.isnull().mean()*100).sort_values(ascending = False).head(4)

# Feature Engineering

## Transforming Numeric Data

There are only four numeric columns in the data set: `age`, `lvesv`, `lvedv`, `lvef`.  Of the four, only `lvef` does not have any kind of a normal distribution: `age` is close to normally distributed, while `lvesv` and `lvedv` are log-normally distributed.

We cannot do anything to `lvedv` because that is my target variable, but we can take the log of `lvesv` (in this case the natural log).  We also tried squaring `age` but that did not affect the distribution in the way we hoped it would.

In [ ]:
# Take the natural log of `lvesv`.
# We chose to make it it's own column rather
# than overwrite the column.

mri["lvesv_log"] = mri["lvesv"].apply(lambda x: np.log(x))

In [ ]:
print(f"The shape of the dataset is: {mri.shape}")

[Top](#Table-Of-Contents)

## Creating Segmental Features

Because the model attempts to predict the end diastolic volume, we want it to be as accurate as it can be.  As part of that, we will try to use many combinations of features in an attempt to achieve high accuracy.

The data have 34 columns that we wish to engineer: a column measuring scarification and a column measuring ischemia.  Because there are so many of them, we felt the need to experiment with how they are passed into the model.  We are unable to create interaction columns, because there are zeros.  Instead, we elected to create segmental columns by summing similar columns together: we will compare the model's performance with the originals and with the segmental columns.

We used this image to guide our create of segmental columns:

<img src = "../Images/cardiac-segmentation-for-cardiac-perfusion-defects.jpg" alt = "Cardiac Segmentation" height = 750 width = 750>

Case courtesy of Dr Hamid Chalian, <a href="https://radiopaedia.org/">Radiopaedia.org</a>. From the case <a href="https://radiopaedia.org/cases/47102">rID: 47102</a>

In [ ]:
# Creating columns based off of the red, blue, green, and yellow
# I'm combining the apex and apical regions because there is only
# one region in the apex.

# New columns for scar tissue

mri["basal_he"]  = mri["ba_he"] + mri["bas_he"] + mri["bis_he"] \
                   + mri["bi_he"] + mri["bil_he"] + mri["bal_he"]
mri["mid_he"]    = mri["ma_he"] + mri["mas_he"] + mri["mis_he"] \
                   + mri["mi_he"] + mri["mil_he"] + mri["mal_he"]
mri["apical_he"] = mri["aa_he"] + mri["as_he"] + mri["ai_he"] \
                   + mri["al_he"] + mri["apex_he"]

# New columns for ischemia

mri["basal_ischemia"]  = mri["ba_ischemia"] + mri["bas_ischemia"] + mri["bis_ischemia"] \
                         + mri["bi_ischemia"] + mri["bil_ischemia"] + mri["bal_ischemia"]
mri["mid_ischemia"]    = mri["ma_ischemia"] + mri["mas_ischemia"] + mri["mis_ischemia"] \
                         + mri["mi_ischemia"] + mri["mil_ischemia"] + mri["mal_ischemia"]
mri["apical_ischemia"] = mri["aa_ischemia"] + mri["as_ischemia"] + mri["ai_ischemia"] \
                         + mri["al_ischemia"]

print(f"The shape of the dataset is: {mri.shape}")

[Top](#Table-Of-Contents)

## Implementing The Pattern Sub-Model Approach

### Creating The Sub-Models

The pattern sub-model approach depends upon the distribution of the missing data.  In order to visualize the patterns, we chose to use `missingno`'s matrix visual.

In [ ]:
msno.matrix(mri.sort_values(by = ["aortic_stenosis", "aortic_reg", 
                                  "mitral_reg", "tricusp_reg"]).iloc[:, 9:13], 
                            figsize = (18,4));

There are some bands on `NaN`s across all four columns, but they are not consistent.

-----

Because there are four columns with missing data, there are 2<sup>4</sup> (16) possible combinations of missing data.  To create each sub-model, we have to filter the data set by each posible combinations of all four columns.

In [ ]:
# Setting aliases so we did not have to type out
# each column name in full.

a = "aortic_stenosis"
b = "aortic_reg"
c = "mitral_reg"
d = "tricusp_reg"

In [ ]:
# Sub-models with only one column of missing data

submodel_a      = mri[mri[a].notnull()]
submodel_b      = mri[mri[b].notnull()]
submodel_c      = mri[mri[c].notnull()]
submodel_d      = mri[mri[d].notnull()]

# Sub-models with 2 columns of missing data

submodel_ab     = mri[(mri[a].notnull()) & (mri[b].notnull())]
submodel_ac     = mri[(mri[a].notnull()) & (mri[c].notnull())]
submodel_ad     = mri[(mri[a].notnull()) & (mri[d].notnull())]
submodel_bc     = mri[(mri[b].notnull()) & (mri[c].notnull())]
submodel_bd     = mri[(mri[b].notnull()) & (mri[d].notnull())]
submodel_cd     = mri[(mri[c].notnull()) & (mri[d].notnull())]

# Sub-models with 3 columns of missing data

submodel_abc    = mri[(mri[a].notnull()) & (mri[b].notnull()) & \
                      (mri[c].notnull())]
submodel_abd    = mri[(mri[a].notnull()) & (mri[b].notnull()) & \
                      (mri[d].notnull())]
submodel_acd    = mri[(mri[a].notnull()) & (mri[c].notnull()) & \
                      (mri[d].notnull())]

# Sub-model with all columns of missing data

submodel_abcd   = mri[(mri[a].notnull()) & (mri[b].notnull()) & \
                      (mri[c].notnull()) & (mri[d].notnull())]

In [ ]:
# We wanted to have an idea of how large each sub-model is
# We used zip because it made it easier for us to print the
# name and shape of each sub-model

list_of_submodels = [submodel_nm, submodel_a, submodel_b, submodel_c,
                     submodel_d, submodel_ab, submodel_ac, submodel_ad,
                     submodel_bc, submodel_bd, submodel_cd, submodel_abc,
                     submodel_abd, submodel_acd, submodel_abcd]

list_of_submodels_str = ['submodel_nm', 'submodel_a', 'submodel_b', 'submodel_c',
                     'submodel_d', 'submodel_ab', 'submodel_ac', 'submodel_ad',
                     'submodel_bc', 'submodel_bd', 'submodel_cd', 'submodel_abc',
                     'submodel_abd', 'submodel_acd', 'submodel_abcd']

for submodel, submodel_str in zip(list_of_submodels,list_of_submodels_str):    
    print(f"The shape of {submodel_str} is {submodel.shape[0]}")

[Top](#Table-Of-Comments)

### Calculating The RMSE Weights

Because each model has a different length, we want to have the larger sub-models count more in the final averaged. To do that we divided the size of the subset by the length of the original set.

In [ ]:
original_len = len(mri)

In [ ]:
def generate_weights(submodels, denominator):
    submodel_weights = []
    for submodel in submodels:
        submodel_weights.append(submodel.shape[0] / denominator)
    return submodel_weights

In [ ]:
weights = generate_weights(list_of_submodels, original_len)

[Top](#Table-Of-Contents)

# Modeling

## Functions

### Modeling Functions

### Evaluation Functions

## Linear Regression

## Ridge

## LASSO